In [1]:
from pyomo.environ import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scripts.data_import import data_import
from scripts.run_optimization_model import run_model

# Read Data

In [2]:
file = 's2'
path = 'data/full/data_' + file + '.xlsx'
data, basis, bess, service, revenue_change = data_import(path)
bess['s_eff'] = 1


In [3]:
# bess['cap_power'] = 4
# bess['cap_energy'] = 16
# bess['s_eff'] = 1

# data['soc_limit']['min'] = 0.1
# data['activation']['reg'] = 0.15
# data['activation']['reg'] = -0.15

# data['activation']['reg_up'] = data['activation']['reg'].apply(lambda x: x if x > 0 else 0)
# data['activation']['reg_down'] = data['activation']['reg'].apply(lambda x: -x if x < 0 else 0)

# # data['activation']['reg_up_activated'] = 1
# # data['activation']['reg_down_activated'] = 1

# data['activation']['reg_up_activated'] = data['activation']['reg'].apply(lambda x: 1 if x > 0 else 0)
# data['activation']['reg_down_activated'] = data['activation']['reg'].apply(lambda x: 1 if x < 0 else 0)


# result_html_name = "result/" + file +"_mipgap0.005" + "_p" + str(bess['cap_power']) + "_e" + str(bess['cap_energy']) + "_l2__"



# Model

In [4]:
def optimize_revenue(initial_soc, data_input, prices, loads, final_soc_target, cap_settings):

    data, basis, bess, service = data_input

    if cap_settings: 
        cap_power = cap_settings[0]
        cap_energy = cap_settings[1]
    else:
        cap_power = bess['cap_power']
        cap_energy = bess['cap_energy']

    price_df = prices

    if service['load'] == 1:
        load_profile = loads.value

    # Parameters
    lamda_min = data['soc_limit']['min']
    lamda_max = data['soc_limit']['max']
    theta = data['schedule']
    beta = data['activation']
    gamma = data['reserve']

    dt = basis['dt']


    Seff = bess['s_eff']
    Ceff = bess['c_eff']
    Deff = bess['d_eff']
    lamda_max_tech = bess['max_soc']
    lamda_min_tech = bess['min_soc']
    eta = bess['daily_cycle']

    # Price Data for the day
    total_time_period = len(price_df)

    T = range(1, total_time_period + 1)
    SOC_T = range(0, total_time_period + 1)

    p_arb = price_df['arb_energy_price']
    p_reg_up_c = price_df['reg_capacity_price']
    p_reg_up_e = price_df['reg_energy_price']
    p_reg_down_c = price_df['reg_down_capacity_price']
    p_reg_down_e = price_df['reg_down_energy_price']
    p_pres_c = price_df['pres_capacity_price']
    p_pres_e = price_df['pres_energy_price']
    p_cres_c = price_df['cres_capacity_price']
    p_cres_e = price_df['cres_energy_price']
    p_ec = price_df['ec_energy_price']
    p_dr_c = price_df['dr_capacity_price']
    p_dr_e = price_df['dr_energy_price']
    p_il_c = price_df['il_capacity_price']
    p_il_e = price_df['il_energy_price']

    # Define the model
    model = ConcreteModel()

    # Variables
    model.SOC = Var(SOC_T, within=NonNegativeReals, bounds = (0, cap_energy))
    model.E_c = Var(T, within=NonNegativeReals, bounds = (0, cap_energy))
    model.E_cr = Var(T, within=NonNegativeReals, bounds = (0, cap_energy))
    model.E_d = Var(T, within=NonNegativeReals, bounds = (0, cap_energy))
    model.E_dr = Var(T, within=NonNegativeReals, bounds = (0, cap_energy))
 
    model.PC_arb = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.PD_arb = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.P_reg = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.PC_reg = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.PD_reg = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.P_pres = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.P_cres = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))

    model.P_ec = Var(T, within=NonNegativeReals)
    model.PL_ec = Var(T, within=NonNegativeReals)
    model.PC_ec = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.PD_L = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.P_dr = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.P_il = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))

    # model.y_ch = Var(T, within=Binary)
    model.y_minus = Var(T, within=Binary)

    model.R_arb = Var(T, within=Reals)
    model.R_reg = Var(T, within=Reals)
    model.R_pres = Var(T, within=Reals)
    model.R_cres = Var(T, within=Reals)
    model.R_ec = Var(T, within=Reals)
    model.R_dr = Var(T, within=Reals)
    model.R_il = Var(T, within=Reals)

    # Objective function
    model.obj = Objective(
        expr= sum(model.R_arb[t] + model.R_reg[t] + model.R_pres[t] + model.R_cres[t] 
                  + model.R_ec[t] + model.R_dr[t] + model.R_il[t] 
                  for t in T),        
        sense=maximize
    )

    def revenue_arbitrage_rule(model, t):
        return model.R_arb[t] == p_arb[t]  * (model.PD_arb[t] - model.PC_arb[t]) * dt
    model.revenue_arbitrage = Constraint(T, rule=revenue_arbitrage_rule)

    def revenue_primary_reserve_rule(model, t):
        return model.R_pres[t] == (p_pres_c[t]  * model.P_pres[t]
                                        + p_pres_e[t] * beta['pres'][t] * model.P_pres[t]) * dt
    model.revenue_preserve = Constraint(T, rule=revenue_primary_reserve_rule)

    def revenue_contingency_reserve_rule(model, t):
        return model.R_cres[t] == (p_cres_c[t]  * model.P_cres[t]
                                        + p_cres_e[t] * beta['cres'][t] * model.P_cres[t]) * dt
    model.revenue_creserve = Constraint(T, rule=revenue_contingency_reserve_rule)

    if service['reg_symmetric'] == 1:
        def reg_constraint_rule(model, t):
            return model.P_reg[t] == model.PC_reg[t]  + model.PD_reg[t] 
        model.reg_up_down = Constraint(T, rule=reg_constraint_rule)

        def revenue_regulation_rule(model, t):
            return model.R_reg[t] == (p_reg_up_c[t]  * model.P_reg[t]
                                            + p_reg_up_e[t] * beta['reg'][t] * model.P_reg[t]) * dt
        model.revenue_regulation = Constraint(T, rule=revenue_regulation_rule)

    else:

        def revenue_regulation_rule(model, t):
            return model.R_reg[t] == (p_reg_up_c[t]  * model.PD_reg[t] * beta['reg_up_activated'][t] 
                                            + p_reg_down_c[t]  * model.PC_reg[t] * beta['reg_down_activated'][t]
                                            + p_reg_up_e[t] * beta['reg_up'][t] * model.PD_reg[t]
                                            - p_reg_down_e[t] * beta['reg_down'][t] * model.PC_reg[t]
                                            ) * dt
        model.revenue_regulation = Constraint(T, rule=revenue_regulation_rule)


    def revenue_energy_charge_rule(model, t):
        if service['load'] == 1:
            return model.R_ec[t] == p_ec[t]  * (load_profile[t] - model.P_ec[t] * dt) 
        else:
            return model.R_ec[t] == 0
    model.revenue_energy_charge = Constraint(T, rule=revenue_energy_charge_rule)

    def revenue_demand_response_rule(model, t):
        return model.R_dr[t] == (p_dr_c[t]  * model.P_dr[t]
                                        + p_dr_e[t] * beta['dr'][t] * model.P_dr[t]) * dt
    model.revenue_demand_response = Constraint(T, rule=revenue_demand_response_rule)

    def revenue_interruptible_load_rule(model, t):
        return model.R_il[t] == (p_il_c[t]  * model.P_il[t]
                                        + p_il_e[t] * beta['il'][t] * model.P_il[t]) * dt
    model.revenue_interruptible_load = Constraint(T, rule=revenue_interruptible_load_rule)


    # Power balance
    def total_charge_rule(model, t):
        return model.PC_arb[t] + model.PC_reg[t] + model.PC_ec[t]  <= cap_power * model.y_minus[t]
    model.total_charge = Constraint(T, rule=total_charge_rule)

    def total_discharge_rule(model, t):
        return model.PD_arb[t] +  model.PD_reg[t] + model.P_pres[t] +  model.P_cres[t] \
                    + model.PD_L[t] + model.P_dr[t] +  model.P_il[t] <= cap_power * (1 - model.y_minus[t])
    model.total_discharge = Constraint(T, rule=total_discharge_rule)

    # Power balance for BTM services
    def total_grid_power_rule(model, t):
        return model.P_ec[t] == model.PL_ec[t] + model.PC_ec[t]
    model.total_grid_power = Constraint(T, rule=total_grid_power_rule)

    if service['load'] == 1:
        def total_load_balance_rule(model, t):
            return (model.PL_ec[t] +  model.PD_L[t] + beta['dr'][t] * model.P_dr[t] + beta['il'][t] * model.P_il[t]) * dt == load_profile[t]
        model.total_load_balance = Constraint(T, rule=total_load_balance_rule)

    # Energy balance
    
    def energy_discharge_rule(model, t):
        return model.E_d[t] == (model.PD_arb[t] + model.PD_L[t]
                                + beta['reg_up'][t] * model.PD_reg[t]
                                + beta['pres'][t] * model.P_pres[t]
                                + beta['cres'][t] * model.P_cres[t]
                                + beta['dr'][t] * model.P_dr[t]
                                + beta['il'][t] * model.P_il[t]
                                ) * dt
    model.energy_discharge = Constraint(T, rule=energy_discharge_rule)

    def energy_discharge_reserve_rule(model, t):
        return model.E_dr[t] == (
                                   (gamma['reg'][t] - beta['reg_up'][t]) * model.PD_reg[t]
                                +  (gamma['pres'][t] - beta['pres'][t]) * model.P_pres[t]
                                +  (gamma['cres'][t] - beta['cres'][t]) * model.P_cres[t]
                                +  (gamma['dr'][t] - beta['dr'][t]) * model.P_dr[t]
                                +  (gamma['il'][t] - beta['il'][t]) * model.P_il[t]
                                ) * dt
    model.energy_discharge_reserve = Constraint(T, rule=energy_discharge_reserve_rule)

    def energy_charge_rule(model, t):
        return model.E_c[t] == (model.PC_arb[t]  
                                + beta['reg_down'][t] * model.PC_reg[t]
                                + model.PC_ec[t]  
                                ) * dt
    model.energy_charge = Constraint(T, rule=energy_charge_rule)

    def energy_charge_reserve_rule(model, t):
        return model.E_cr[t] == ((gamma['reg'][t] - beta['reg_down'][t]) * model.PC_reg[t]) * dt
    model.energy_charge_reserve = Constraint(T, rule=energy_charge_reserve_rule)


    def soc_constraints(model, t):
        if t == 0:
            return model.SOC[t] == initial_soc
        else:
            return model.SOC[t] == model.SOC[t-1] * Seff + model.E_c[t] * Ceff - model.E_d[t]/Deff
    model.SOC_constraints = Constraint(SOC_T, rule=soc_constraints)

    def soc_lower_limit_constraint(model, t):
        return model.SOC[t] - model.E_dr[t]/Deff >= cap_energy * lamda_min[t]
    model.SOC_lower_limit = Constraint(T, rule=soc_lower_limit_constraint)

    def soc_upper_limit_constraint(model, t):
        return model.SOC[t] + model.E_cr[t] * Ceff <= cap_energy * lamda_max[t]
    model.SOC_upper_limit = Constraint(T, rule=soc_upper_limit_constraint)

    if final_soc_target:
        def soc_final_state_constraint(model):
            end_t = SOC_T[-1]
            return model.SOC[end_t] == bess['initial_soc'] * cap_energy
        model.SOC_final = Constraint(rule=soc_final_state_constraint)


    # Cycle limits

    if service['reg_activate'] != 1:

        def cycle_charge_constraint(model):
            return sum(model.PC_arb[t] + model.PC_ec[t] +  model.PC_reg[t] * gamma['reg'][t] for t in T)*dt <= eta * cap_energy * (lamda_max_tech - lamda_min_tech)
        model.cycle_charge = Constraint(rule=cycle_charge_constraint)

        def cycle_discharge_constraint(model):
            return sum(model.PD_arb[t] + model.PD_L[t] + model.PD_reg[t] * gamma['reg'][t] for t in T)*dt <= eta * cap_energy * (lamda_max_tech - lamda_min_tech)
        model.cycle_discharge = Constraint(rule=cycle_discharge_constraint)

    else:

        def cycle_charge_constraint(model):
            return sum(model.PC_arb[t] + model.PC_ec[t] +  model.PC_reg[t] * beta['reg_down'][t] for t in T)*dt <= eta * cap_energy * (lamda_max_tech - lamda_min_tech)
        model.cycle_charge = Constraint(rule=cycle_charge_constraint)

        def cycle_discharge_constraint(model):
            return sum(model.PD_arb[t] + model.PD_L[t] + model.PD_reg[t] * beta['reg_up'][t] for t in T)*dt <= eta * cap_energy * (lamda_max_tech - lamda_min_tech)
        model.cycle_discharge = Constraint(rule=cycle_discharge_constraint)


    # Service availability

    def arbitrage_availability_rule(model, t):
        return model.PC_arb[t] + model.PD_arb[t] <= cap_power * service['arb'] * theta['arb'].loc[t] * 999999
    model.arbitrage_availability = Constraint(T, rule=arbitrage_availability_rule)

    def regulation_availability_rule(model, t):
        return model.PC_reg[t] + model.PD_reg[t] <= cap_power * service['reg'] * theta['reg'].loc[t] * 999999
    model.regulation_availability = Constraint(T, rule=regulation_availability_rule)

    def preserve_availability_rule(model, t):
        return model.P_pres[t] <= cap_power * service['pres'] * theta['pres'].loc[t] * 999999
    model.preserve_availability = Constraint(T, rule=preserve_availability_rule)

    def creserve_availability_rule(model, t):
        return model.P_cres[t] <= cap_power * service['cres'] * theta['cres'].loc[t] * 999999
    model.creserve_availability = Constraint(T, rule=creserve_availability_rule)

    def energy_charge_availability_rule(model, t):
        return model.P_ec[t] <= cap_power * service['ec'] * theta['ec'].loc[t] * 999999
    model.energy_charge_availability = Constraint(T, rule=energy_charge_availability_rule)

    def demand_response_availability_rule(model, t):
        return model.P_dr[t] <= cap_power * service['dr'] * theta['dr'].loc[t] * 999999
    model.demand_response_availability = Constraint(T, rule=demand_response_availability_rule)

    def interruptible_load_availability_rule(model, t):
        return model.P_il[t] <= cap_power * service['il'] * theta['il'].loc[t] * 999999
    model.interruptible_load_availability = Constraint(T, rule=interruptible_load_availability_rule)


    # Solve the model
    solver = SolverFactory('glpk')
    solver.options['mipgap'] = 0.005  
    solver.options['tmlim'] = 10

    solver.solve(model, tee=True)

    schedule_dict = {}
    revenue_dict = {}

    # Extract results for charging, discharging, and SOC
    schedule_dict['arb_charge'] = [model.PC_arb[t].value for t in T]  
    schedule_dict['arb_discharge'] = [model.PD_arb[t].value for t in T]
    schedule_dict['reg_down'] = [model.PC_reg[t].value for t in T] 
    schedule_dict['reg_up'] = [model.PD_reg[t].value for t in T]  
    schedule_dict['pres'] = [model.P_pres[t].value for t in T]  
    schedule_dict['cres'] = [model.P_cres[t].value for t in T]  
    schedule_dict['dr'] = [model.P_dr[t].value for t in T] 
    schedule_dict['il'] = [model.P_il[t].value for t in T] 
    schedule_dict['grid_purchase'] = [model.P_ec[t].value for t in T]  
    schedule_dict['grid_to_storage'] = [model.PC_ec[t].value for t in T]  
    schedule_dict['grid_to_load'] = [model.PL_ec[t].value for t in T] 
    schedule_dict['storage_to_load'] = [model.PD_L[t].value for t in T]  
    schedule_dict['soc'] = [model.SOC[t].value for t in SOC_T]  

    revenue_dict['arb'] =  [model.R_arb[t]() for t in T] 
    revenue_dict['reg'] = [model.R_reg[t]() for t in T ]
    revenue_dict['pres'] = [model.R_pres[t]() for t in T ]
    revenue_dict['cres'] = [model.R_cres[t]() for t in T] 
    revenue_dict['dr'] = [model.R_dr[t]() for t in T ]
    revenue_dict['il'] = [model.R_il[t]() for t in T ]
    revenue_dict['ec'] = [model.R_ec[t]() for t in T]

    # Return the results and final SOC
    return model, model.obj(), schedule_dict, revenue_dict

# RUN

In [5]:
power_range = list(range(1, 17))
energy_range = list(range(1, 17))

econs_sens_keys = []
econs_sens_values = []

for power in power_range:
    for energy in energy_range:

        power_target = power * 0.5
        energy_target = energy * 0.5

        print("Solving: ", power_target, energy_target)

        bess['cap_power'] = power_target
        bess['cap_energy'] = energy_target

        packaged_data = [data, basis, bess, service] 

        annual_revenue, result_df, revenue_df, financial_metrics = run_model(packaged_data, result_html_name = [])

        [NPV, ROI, IRR, BCR, PP] = financial_metrics

        econs_sens_keys.append([power_target, energy_target])
        econs_sens_values.append([annual_revenue] + financial_metrics)


Solving:  0.5 0.5
Solving:  0.5 1.0
Solving:  0.5 1.5
Solving:  0.5 2.0
Solving:  0.5 2.5
Solving:  0.5 3.0
Solving:  0.5 3.5
Solving:  0.5 4.0
Solving:  0.5 4.5
Solving:  0.5 5.0
Solving:  0.5 5.5
Solving:  0.5 6.0
Solving:  0.5 6.5
Solving:  0.5 7.0
Solving:  0.5 7.5
Solving:  0.5 8.0
Solving:  1.0 0.5
Solving:  1.0 1.0
Solving:  1.0 1.5
Solving:  1.0 2.0
Solving:  1.0 2.5
Solving:  1.0 3.0
Solving:  1.0 3.5
Solving:  1.0 4.0
Solving:  1.0 4.5
Solving:  1.0 5.0
Solving:  1.0 5.5
Solving:  1.0 6.0
Solving:  1.0 6.5
Solving:  1.0 7.0
Solving:  1.0 7.5
Solving:  1.0 8.0
Solving:  1.5 0.5
Solving:  1.5 1.0
Solving:  1.5 1.5
Solving:  1.5 2.0
Solving:  1.5 2.5
Solving:  1.5 3.0
Solving:  1.5 3.5
Solving:  1.5 4.0
Solving:  1.5 4.5
Solving:  1.5 5.0
Solving:  1.5 5.5
Solving:  1.5 6.0
Solving:  1.5 6.5
Solving:  1.5 7.0
Solving:  1.5 7.5
Solving:  1.5 8.0
Solving:  2.0 0.5
Solving:  2.0 1.0
Solving:  2.0 1.5
Solving:  2.0 2.0
Solving:  2.0 2.5
Solving:  2.0 3.0
Solving:  2.0 3.5
Solving:  

In [6]:
df = pd.DataFrame(index = pd.MultiIndex.from_tuples(econs_sens_keys, names = ['Power', 'Energy']), data = econs_sens_values, columns = ['annual_revenue', 'NPV', 'ROI', 'IRR', 'BCR', 'PP'])
df['annual_revenue_per_kw'] = df['annual_revenue']/df.index.get_level_values(0)/1000
df = df.reset_index()
# df['Energy'] = (df['Energy'] * 0.5) * df['Power']
# df = df.set_index(['Power', 'Energy'])

In [7]:
pd.set_option('display.max_rows', None)
df

,Power,Energy,annual_revenue,NPV,ROI,IRR,BCR,PP,annual_revenue_per_kw
0,0.5,0.5,1.228977e+05,5.526996e+05,1.394706,0.297212,2.394706,4.0,245.795387
1,0.5,1.0,1.228977e+05,2.823096e+05,0.423460,0.132671,1.423460,7.0,245.795387
2,0.5,1.5,1.228977e+05,1.191956e+04,0.012720,0.052675,1.012720,10.0,245.795387
3,0.5,2.0,1.228977e+05,-2.584704e+05,-0.214062,0.002276,0.785938,NaN,245.795387
4,0.5,2.5,1.228977e+05,-5.288604e+05,-0.357859,-0.033645,0.642141,NaN,245.795387
5,0.5,3.0,1.228977e+05,-7.992504e+05,-0.457176,-0.061158,0.542824,NaN,245.795387
6,0.5,3.5,1.228977e+05,-1.069640e+06,-0.529886,-0.083249,0.470114,NaN,245.795387
7,0.5,4.0,1.228977e+05,-1.340030e+06,-0.585418,-0.101585,0.414582,NaN,245.795387
8,0.5,4.5,1.228977e+05,-1.610420e+06,-0.629217,-0.117182,0.370783,NaN,245.795387
9,0.5,5.0,1.228977e+05,-1.880810e+06,-0.664646,-0.130705,0.335354,NaN,245.795387


In [16]:
pivot_df = df.pivot(index="Energy", columns="Power", values="annual_revenue_per_kw") * 100
pivot_df

Power,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,5.5,6.0,6.5,7.0,7.5,8.0
Energy,,,,,,,,,,,,,,,,
0.5,24579.538722,16396.960473,12424.437548,9962.519014,8224.754929,7180.787639,6202.337875,5324.905957,4445.024499,2756.319017,2505.744561,2296.932514,2120.245398,1968.799298,1837.546011,1722.699386
1.0,24579.538722,24579.538722,19644.656236,16396.960473,14119.086105,12424.354752,11002.869602,9962.308636,8954.142581,8224.846884,7650.128644,7180.661906,6666.992716,6202.402871,5745.330706,5324.891120
1.5,24579.538722,24579.538722,24579.538722,21095.432768,18397.789016,16396.960473,14805.596266,13499.874276,12425.601798,11419.668993,10619.383458,9961.588290,9264.513186,8682.488553,8225.177698,7830.283495
2.0,24579.538722,24579.538722,24579.538722,24579.538722,21873.651949,19644.656236,17834.079701,16396.960473,15155.316568,14119.456466,13218.505698,12430.961775,11647.700698,11003.874451,10436.888257,9963.791467
2.5,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,22416.469845,20462.211016,18862.873923,17520.496118,16396.960473,15382.398824,14541.052500,13737.729893,13056.304896,12431.168773,11785.879763
3.0,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,22751.941890,21095.432768,19644.656236,18395.556638,17311.247652,16396.960473,15534.727683,14805.500376,14119.005990,13502.862069
3.5,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,23011.707256,21531.931354,20222.002616,19067.789202,18086.507323,17172.047610,16396.960473,15648.296188,15001.596792
4.0,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,23206.607891,21873.651949,20696.804777,19644.656236,18681.974565,17834.510322,17070.844404,16396.960473
4.5,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,24579.538722,23348.909980,22159.554321,21095.432768,20088.879716,19196.697466,18397.289254,17661.059096


In [17]:
import plotly.graph_objects as go

# Data for the heatmap
z = pivot_df.values
x = pivot_df.columns
y = pivot_df.index

# Creating the heatmap
fig = go.Figure(data=go.Heatmap(
    z=z,
    x=x,
    y=y,
    colorscale='RdBu',
    reversescale=True,  
    colorbar=dict(title='')
))

# Adding titles and labels
fig.update_layout(
    # title='Heatmap of ROI with varying Power and Energy (Reversed Energy Axis)',
    xaxis=dict(title='Power'),
    yaxis=dict(title='Energy')
)

# Display the figure
fig.show()


In [18]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Calculating Energy/Power ratio for the contour plot
df['Ratio'] = df['Energy'] / df['Power']
pivot_ratio = df.pivot(index="Energy", columns="Power", values="Ratio")

# Data for the heatmap
z = pivot_df.values
x = pivot_df.columns
y = pivot_df.index

# Creating the heatmap with contour lines
fig = go.Figure()

# Add heatmap
fig.add_trace(go.Heatmap(
    z=z,
    x=x,
    y=y,
    colorscale='RdBu',
    reversescale=True,  # High values red, low values blue
    colorbar=dict(title='ROI (%)'),
    zmin=np.min(z),  # Define the range of z for better contrast
    zmax=np.max(z)
))

# Add contour lines for the Energy/Power ratio with text labels
fig.add_trace(go.Contour(
    z=pivot_ratio.values,
    x=pivot_df.columns,
    y=pivot_df.index,
    showscale=False,  # Hide the scale for contour lines
    contours=dict(
        coloring='lines',
        showlabels=True,  # Enable labels on contours
        labelfont=dict(size=12, color='black'),  # Customize the font of the labels
        start=0.5,  # Starting value for contour lines
        end=4,     # Ending value for contour lines
        size=0.5   # Interval between contour lines
    ),
    line=dict(color='black', width=2),
    # ncontours=50  # Increase the number of contour lines

))

# Adding titles and labels, and setting axis ranges to start from 1
fig.update_layout(
    title='Heatmap of ROI with Energy/Power Ratio Contour Lines',
    xaxis=dict(title='Power', range=[0.5, 8]),
    yaxis=dict(title='Energy', range=[0.5, 8])
)

# Display the figure
fig.show()
